In [ ]:
from cortexlib.utils.random import set_global_seed

set_global_seed()

In [ ]:
from cortexlib.utils.file import read_json_file_as_dataframe

# For each layer, results has an FEV score for predicting neural data from SimCLR features reduced to N PCs
# Load results and filter the best FEV per layer (e.g. feats reduced to optimal number of PCs)
fev_results = read_json_file_as_dataframe('../regressions/results/simclr_feats_pcs_to_neural_data.json')
best_fev_per_layer = fev_results.loc[fev_results.groupby("layer")["mean_fev"].idxmax()]

best_fev_per_layer

In [ ]:
import torch
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    simclr_data = torch.load('../regressions/features/simclr_features_all_layers.pt')

simclr_feats = simclr_data['features']

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    neural_data = torch.load('../regressions/neural_data/neural_responses_mean.pt')

neural_responses_mean = neural_data['neural_responses_mean']

In [ ]:
from cortexlib.rsa import RSA
import pandas as pd
from sklearn.decomposition import PCA
from tqdm import tqdm

rsa = RSA(neural_data=neural_responses_mean, neural_data_pc_index=0)

rsa_results = []

for layer, feats in tqdm(simclr_feats.items()):
    feats = feats if feats.dim() <= 2 else feats.view(feats.size(0), -1)
    
    for n_pcs in [None, 10, 20, 50, 100, 200, 300, 400, 500]:
        feats_pca = feats if n_pcs is None else PCA(n_pcs).fit_transform(feats)
        sim = rsa.compute_similarity_with_neural_data(feats_pca)
        rsa_results.append({
            'layer': layer,
            'n_pcs': n_pcs,
            'spearman_correlation': sim,
        })

In [ ]:
from cortexlib.utils.file import write_json_file

write_json_file(rsa_results, './results/rsa_simclr_feats_pcs_vs_pc1_neural_data.json')

In [ ]:
rsa_results_df = pd.DataFrame(rsa_results)

best_rsa_per_layer = rsa_results_df.loc[
    rsa_results_df.groupby('layer')['spearman_correlation'].idxmax()
].reset_index(drop=True)

best_rsa_per_layer

In [ ]:
merged_results = pd.merge(best_fev_per_layer, best_rsa_per_layer, on='layer')
merged_results

In [ ]:
import matplotlib.pyplot as plt
from cortexlib.utils.plotting import simclr_colours
from adjustText import adjust_text

# TODO: compose into plotting function

colours = merged_results['layer'].map(simclr_colours)

plt.figure(figsize=(10, 6))
plt.scatter(merged_results['spearman_correlation'], merged_results['mean_fev'], c=colours)

texts = []
for _, row in merged_results.iterrows():
    texts.append(
        plt.text(row['spearman_correlation'], row['mean_fev'], row['layer'],
                 color=simclr_colours[row['layer']], fontsize=10)
    )

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray'))

plt.xlabel('RSA (Spearman Correlation)')
plt.ylabel('Mean FEV')
plt.title('SimCLR Layers: RSA vs. Predictive Power (PC1 Neural Data)')
plt.grid(True)
plt.tight_layout()
plt.show()